In [1]:
import pandas as pd

In [2]:
import numpy as np

In [21]:
df = pd.read_csv('train_delivery_data.csv', header=None)

In [4]:
df.columns = ['deliverer_id', 'delivery_zone', 'monthly_app_usage','subscription_type', 'paid_price','customer_size', 'menu','delay_time']


In [5]:
df.head()

,deliverer_id,delivery_zone,monthly_app_usage,subscription_type,paid_price,customer_size,menu,delay_time
0,0,VI,17,Semestral,16.940991,4,French,11.332126
1,98,VII,18,Free,39.064177,5,Asian,10.429260
2,14,IV,16,Trimestral,32.826544,2,Italian,7.900892
3,72,V,17,Free,7.556762,3,Mexican,14.689806
4,11,VIII,12,Semestral,34.200684,1,Indian,12.513841


In [6]:
media_delay = np.mean(df.delay_time)
media_delay

9.87754104229854

In [7]:
df['delay_time'] = np.where(df['delay_time']>media_delay, 1, 0)
df.head()

,deliverer_id,delivery_zone,monthly_app_usage,subscription_type,paid_price,customer_size,menu,delay_time
0,0,VI,17,Semestral,16.940991,4,French,1
1,98,VII,18,Free,39.064177,5,Asian,1
2,14,IV,16,Trimestral,32.826544,2,Italian,0
3,72,V,17,Free,7.556762,3,Mexican,1
4,11,VIII,12,Semestral,34.200684,1,Indian,1


In [8]:
df = pd.get_dummies(df,columns=['subscription_type','delivery_zone','menu'],drop_first=True)

In [9]:
df.head()

,deliverer_id,monthly_app_usage,paid_price,customer_size,delay_time,subscription_type_Monthly,subscription_type_Prepaid,subscription_type_Semestral,subscription_type_Trimestral,subscription_type_Yearly,...,delivery_zone_IV,delivery_zone_V,delivery_zone_VI,delivery_zone_VII,delivery_zone_VIII,menu_French,menu_Indian,menu_Italian,menu_Japanese,menu_Mexican
0,0,17,16.940991,4,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
1,98,18,39.064177,5,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,14,16,32.826544,2,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
3,72,17,7.556762,3,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,11,12,34.200684,1,1,0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
y = df.pop('delay_time')


In [12]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=11238)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB

In [14]:
logi = LogisticRegression().fit(X_train, y_train)
Ber = BernoulliNB().fit(X_train, y_train)
Desi = DecisionTreeClassifier().fit(X_train, y_train)
rand = RandomForestClassifier().fit(X_train, y_train)

/Users/moisesquiroz/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/moisesquiroz/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [15]:
from sklearn.metrics import classification_report

In [16]:
modelos = [logi,Ber,Desi,rand]
text_file = open("candidate_models.txt", "w")
for mo in modelos:
    text_file.write("modelo: "+type(mo).__name__+'\n')
    print("modelo:",type(mo).__name__)
    text_file.write(classification_report(y_test,mo.predict(X_test)))
    print(classification_report(y_test,mo.predict(X_test)))
text_file.close()

modelo: LogisticRegression
              precision    recall  f1-score   support

           0       0.46      0.56      0.51       167
           1       0.42      0.32      0.36       163

    accuracy                           0.44       330
   macro avg       0.44      0.44      0.43       330
weighted avg       0.44      0.44      0.43       330

modelo: BernoulliNB
              precision    recall  f1-score   support

           0       0.51      0.60      0.55       167
           1       0.50      0.42      0.46       163

    accuracy                           0.51       330
   macro avg       0.51      0.51      0.50       330
weighted avg       0.51      0.51      0.51       330

modelo: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.52      0.44      0.47       167
           1       0.50      0.58      0.54       163

    accuracy                           0.51       330
   macro avg       0.51      0.51      0.51       

In [17]:
from sklearn.externals import joblib

/Users/moisesquiroz/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [18]:
filename = 'DecisionTreeClassifier.pkl'
joblib.dump(Desi, filename)
print("mejor modelo DesionTree")

mejor modelo DesionTree


In [ ]:
joblib.load()